In [1]:
import sys
sys.path.append('../../')

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

In [2]:
spark = SparkSession.builder \
    .appName("ner")\
    .master("local[1]")\
    .config("spark.driver.memory","4G")\
    .config("spark.driver.maxResultSize", "2G")\
    .config("spark.jar", "lib/sparknlp.jar")\
    .config("spark.kryoserializer.buffer.max", "500m")\
    .getOrCreate()

1. Download CoNLL2003 dataset
2. Save 3 files eng.train, eng.testa, eng.testa, into working dir ./

In [3]:
from pyspark.sql.types import *

class Annotation:
    def __init__(self, annotatorType, begin, end, metadata):
        self.annotatorType = annotatorType
        self.begin = begin
        self.end = end
        self.metadata = metadata

        
annotation_schema = StructType([
    StructField("annotatorType", StringType()),
    StructField("begin", IntegerType(), False),
    StructField("end", IntegerType(), False),
    StructField("metadata", MapType(StringType(), StringType()))
])
    


def readDataset(file, doc_column = "text", label_column = "label"):
    global spark
    
    result = []
    doc = ""
    labels = []

    with open(file) as f:
        for line in f:
            items = line.split(' ')
            word = items[0]
            if word == "-DOCSTART-":
                result.append((doc, labels))
                doc = ""
                labels = []
            elif len(items) <= 1:
                doc = doc + " \n"
            else:
                if len(doc) > 0:
                    doc = doc + " "

                begin = len(doc)
                doc = doc + word
                end = len(doc) - 1
                ner = items[3]
                labels.append(Annotation("named_entity", begin, end, {"tag": ner}))

    if doc:
        result.append((doc, labels))
    
    global annotation_schema
    
    schema =  StructType([
      StructField(doc_column, StringType()),
      StructField(label_column, ArrayType(annotation_schema))
    ])
    
    
    return spark.createDataFrame(result, schema = schema)

In [5]:
import time

def get_pipeline():
    documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

    sentenceDetector = SentenceDetectorModel()\
      .setInputCols(["document"])\
      .setOutputCol("sentence")

    tokenizer = RegexTokenizer()\
      .setInputCols(["document"])\
      .setOutputCol("token")

    posTagger = PerceptronApproach()\
      .setCorpusPath("../../../src/test/resources/anc-pos-corpus/")\
      .setIterations(5)\
      .setInputCols(["token", "document"])\
      .setOutputCol("pos")

    nerTagger = CrfBasedNer()\
      .setInputCols(["sentence", "token", "pos"])\
      .setLabelColumn("label")\
      .setOutputCol("ner")\
      .setMinEpochs(1)\
      .setMaxEpochs(300)\
      .setLossEps(1e-3)\
      .setDicts(["../../../src/main/resources/ner-corpus/dict.txt"])\
      .setL2(1)\
      .setC0(1250000)\
      .setRandomSeed(100)\
      .setVerbose(2)
      
    pipeline = Pipeline(
        stages = [
        documentAssembler,
        sentenceDetector,
        tokenizer,
        posTagger,
        nerTagger
      ])
    
    return pipeline


def train_model(file):
    global spark
    
    print("Dataset Reading")
    
    start = time.time()
    dataset = readDataset(file)
    print("Done, {}\n".format(time.time() - start))

    print("Start fitting")
    pipeline = get_pipeline()

    return pipeline.fit(dataset)

In [6]:
from pyspark.sql.functions import col, udf, explode


def get_dataset_for_analysis(file, model):
    global spark
    
    print("Dataset Reading")
    
    start = time.time()
    dataset = readDataset(file)
    print("Done, {}\n".format(time.time() - start))
    
    predicted = model.transform(dataset)
    
    global annotation_schema
    
    zip_annotations = udf(
      lambda x, y: list(zip(x, y)),
      ArrayType(StructType([
          StructField("predicted", annotation_schema),
          StructField("label", annotation_schema)
      ]))
    )
    
    return predicted\
        .withColumn("result", zip_annotations("ner", "label"))\
        .select(explode("result").alias("result"))\
        .select(
            col("result.predicted").alias("predicted"), 
            col("result.label").alias("label")
        )
        
def printStat(label, correct, predicted, predictedCorrect):
    prec = predictedCorrect / predicted if predicted > 0 else 0
    rec = predictedCorrect / correct if correct > 0 else 0
    f1 = (2*prec*rec)/(prec + rec) if prec + rec > 0 else 0
    
    print("{}\t{}\t{}\t{}".format(label, prec, rec, f1))
        

def test_dataset(file, model, ignore_tokenize_misses=True):
    global spark
    
    started = time.time()

    df = readDataset(file)
    transformed = model.transform(df).select("label", "ner")

    labels = []
    predictedLabels = []

    for line in transformed.collect():
        label = line[0]
        ner = line[1]
    
        ner = {(a["begin"], a["end"]):a["metadata"]["tag"] for a in ner}

        for a in label:
            key = (a["begin"], a["end"])

            label = a["metadata"]["tag"].strip()
            predictedLabel = ner.get(key, "O").strip()
            
            if key not in ner and ignore_tokenize_misses:
                continue
                
            labels.append(label)
            predictedLabels.append(predictedLabel)
        

    correct = {}
    predicted = {}
    predictedCorrect = {}


    print(len(labels))

    for (lPredicted, lCorrect) in zip(predictedLabels, labels):
        correct[lCorrect] = correct.get(lCorrect, 0) + 1
        predicted[lPredicted] = predicted.get(lPredicted, 0) + 1

        if lCorrect == lPredicted:
            predictedCorrect[lPredicted] = predictedCorrect.get(lPredicted, 0) + 1

    correct = { key: correct[key] for key in correct.keys() if key != 'O'}
    predicted = { key: predicted[key] for key in predicted.keys() if key != 'O'}
    predictedCorrect = { key: predictedCorrect[key] for key in predictedCorrect.keys() if key != 'O'}

    tags = set(list(correct.keys()) + list(predicted.keys()))

    print("label\tprec\trec\tf1")
    totalCorrect = sum(correct.values())
    totalPredicted = sum(predicted.values())
    totalPredictedCorrect = sum(predictedCorrect.values())

    printStat("Total", totalCorrect, totalPredicted, totalPredictedCorrect)

    for label in tags:
        printStat(label, correct.get(label, 0), predicted.get(label, 0), predictedCorrect.get(label, 0))


In [7]:
import os.path

folder = '.'
train_file = os.path.join(folder, "eng.train")
test_file_a = os.path.join(folder, "eng.testa")
test_file_b = os.path.join(folder, "eng.testb")

In [8]:
model = train_model(train_file)

Dataset Reading
Done, 8.26949167251587

Start fitting


In [9]:
print("\nQuality on training data")
test_dataset(train_file, model)

print("\n\nQuality on validation data")
test_dataset(test_file_a, model)

print("\n\nQuality on test data")
test_dataset(test_file_b, model)


Quality on training data
193362
label	prec	rec	f1
Total	0.9993112085159674	0.9990922465333208	0.999201715528981
I-MISC	0.9983788791106994	0.9983788791106994	0.9983788791106994
I-LOC	0.9989577905158937	0.9990879478827361	0.9990228649599375
B-MISC	1.0	0.8648648648648649	0.927536231884058
B-LOC	1.0	1.0	1.0
I-ORG	0.9997889627519257	0.9990510333192746	0.9994198618216339
B-ORG	1.0	1.0	1.0
I-PER	0.9995193232070756	0.9999038276591652	0.9997115384615385


Quality on validation data
48717
label	prec	rec	f1
Total	0.9134639857470094	0.8904602406649299	0.9018154406683837
I-MISC	0.9235127478753541	0.8056013179571664	0.860536735591729
B-MISC	0.0	0.0	0
I-ORG	0.8752566735112937	0.8559236947791165	0.8654822335025382
I-LOC	0.9134266459305339	0.9129533678756476	0.913189945581757
I-PER	0.9356384799726121	0.9356384799726121	0.9356384799726121


Quality on test data
44050
label	prec	rec	f1
Total	0.8441830065359477	0.8516418304101279	0.8478960152300926
I-MISC	0.7660332541567696	0.7448036951501155	0.755269320

In [ ]:
df = get_dataset_for_analysis(test_file_a, model, spark)
df.show()

In [10]:
get_pipeline().write().overwrite().save("./crf_pipeline")
model.write().overwrite().save("./crf_model")

In [11]:
from pyspark.ml import PipelineModel, Pipeline

Pipeline.read().load("./crf_pipeline")
sameModel = PipelineModel.read().load("./crf_model")

In [12]:
print("\nQuality on training data")
test_dataset(train_file, sameModel)

print("\n\nQuality on validation data")
test_dataset(test_file_a, sameModel)

print("\n\nQuality on test data")
test_dataset(test_file_b, sameModel)


Quality on training data
193362
label	prec	rec	f1
Total	0.9993112085159674	0.9990922465333208	0.999201715528981
I-MISC	0.9983788791106994	0.9983788791106994	0.9983788791106994
I-LOC	0.9989577905158937	0.9990879478827361	0.9990228649599375
B-MISC	1.0	0.8648648648648649	0.927536231884058
B-LOC	1.0	1.0	1.0
I-ORG	0.9997889627519257	0.9990510333192746	0.9994198618216339
B-ORG	1.0	1.0	1.0
I-PER	0.9995193232070756	0.9999038276591652	0.9997115384615385


Quality on validation data
48717
label	prec	rec	f1
Total	0.9134639857470094	0.8904602406649299	0.9018154406683837
I-MISC	0.9235127478753541	0.8056013179571664	0.860536735591729
B-MISC	0.0	0.0	0
I-ORG	0.8752566735112937	0.8559236947791165	0.8654822335025382
I-LOC	0.9134266459305339	0.9129533678756476	0.913189945581757
I-PER	0.9356384799726121	0.9356384799726121	0.9356384799726121


Quality on test data
44050
label	prec	rec	f1
Total	0.8441830065359477	0.8516418304101279	0.8478960152300926
I-MISC	0.7660332541567696	0.7448036951501155	0.755269320